In [317]:
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import *
from typing import *
import scipy.integrate as integrate
import unittest 
import timeit
import random
import sys
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from EnforceTyping import enforce_method_typing, EnforceClassTyping
from MDPFramework import MDPEnvironment, MDPController, LearningAgent

In [318]:
@dataclass(unsafe_hash=True, order=True)
class Node(EnforceClassTyping):
  id: Union[str, int, float, np.ndarray]
  neighbors: Dict
  def __repr__(self) -> str:
    neighbor_ids= tuple(neighbor.id for neighbor in self.neighbors['Nodes'])
    name= str(self.id)+ ':'+ str(neighbor_ids)
    return name
  
  def add_neighbor(self, neighbor: 'Node', ):
    """
    Adds a neighbor to the node's list of neighbors.
    """
    self.neighbors['Nodes'].append(neighbor) 
    self.neighbors['Nodes'].append(neighbor) 

@dataclass(kw_only=True)
class NodeNetwork(EnforceClassTyping):
  n_nodes: int
  n_connections: int
  network: Tuple[Node]= ()

  @property
  def transition_probability(self, node1: Node):
    pass
  def __post_init__(self):
    nodes= self.generate_states_space(self.n_nodes)
    self.network= self.randomly_connect_state_space(nodes, self.n_connections)
  
  def add_node(self, node: Node):
    """
    Adds a new node to the network.
    """
    self.network= self.network+ (node, )

  def generate_states_space(self, n_nodes: int)-> Tuple[Node]:
    nodes= ()
    for i in range(n_nodes):
      nodes= nodes+ (Node(id= i, neighbors={"Nodes": [ ],
                                            "TransitionProbablilities": [ ]}), )
    return nodes

  def randomly_connect_state_space(self, nodes: Tuple[Node], n_connections: int)-> Tuple[Node]:
    for _ in range(n_connections):
      random_state1, random_state2= random.sample(sorted(nodes), 2)
      if not self.are_connected(random_state1, random_state2):
          self.connect_nodes(random_state1, random_state2)
    return nodes

  def connect_nodes(self, node1: Node, node2: Node):
    """
    Connects two nodes in the network by adding each other to their respective neighbor lists.
    """
    if self.are_connected(node1, node2):
      return
    node1.add_neighbor(node2)
    node2.add_neighbor(node1)

  def are_connected(self, node1: Node, node2: Node):
     are_connected= node1 in node2.neighbors["Nodes"] and node2 in node1.neighbors["Nodes"]
     return are_connected


@dataclass(kw_only=True)
class FiniteStateAutomaton(MDPEnvironment):
  n_states: int
  
  @dataclass(unsafe_hash=True, order=True)
  class State:
    """
    Represents a single node in the network.
    """
    id: Union[str, int, float, np.ndarray]
    neighbors: tuple

    def __repr__(self) -> str:
      neighbor_ids= tuple(neighbor.id for neighbor in self.neighbors)
      name= str(self.id)+ ':'+ str(neighbor_ids)
      return name
    
    def add_neighbor(self, neighbor: 'State'):
      """
      Adds a neighbor to the node's list of neighbors.
      """
      self.neighbors = self.neighbors+ (neighbor,) 

  state_space: Tuple[State]= None
  initial_state: State= None
  current_state: State= None

  @property
  def transition_probability(self, state: State):
    pass

  def __post_init__(self):
    states= self.generate_states_space(self.n_states)
    self.state_space= self.randomly_connect_state_space(states, 5)
    self.initial_state= random.choice(self.state_space)
    self.current_state= self.initial_state
  
  def add_node(self, node: State):
    """
    Adds a new node to the network.
    """
    self.state_space= self.state_space+ (node, )

  def generate_states_space(self, n_states: int)-> Tuple[State]:
    state_space= ()
    for i in range(n_states):
      state_space= state_space+ (self.State(id= i, neighbors=()), )
    return state_space

  def randomly_connect_state_space(self, state_space: Tuple[State], n_connections: int)-> Set[State]:
    for _ in range(n_connections):
      random_state1, random_state2= random.sample(sorted(state_space), 2)
      if not self.are_connected(random_state1, random_state2):
          self.connect_nodes(random_state1, random_state2)
    return state_space

  def connect_nodes(self, state1: State, state2: State):
    """
    Connects two nodes in the network by adding each other to their respective neighbor lists.
    """
    state1.add_neighbor(state2)
    state2.add_neighbor(state1)

  def are_connected(self, state1: State, state2: State):
     are_connected= state1 in state2.neighbors and state2 in state1.neighbors
     return are_connected

  def state_transition_model(self, state: State, action: np.ndarray)-> State:
    if len(state.neighbors)> 1:
      rand_segments= np.random.random((len(state.neighbors)-1, ))   

  def reward_model(self, state: State, action: np.ndarray, next_state: State, terminal_signal: bool)-> float:
    '''This is a scalar performance metric.'''
    ...

  def is_terminal_condition(self, state: State)-> bool:
    if len(state.neighbors) == 0:
      return True

  def transition_step(self, state: State, action: np.ndarray)-> tuple[State, float, bool]:
    ...

  def sample_trajectory(self, runtime: float)-> list[State]:
    ...


In [319]:
test_graph= NodeNetwork(n_nodes=10, n_connections=10)

test_node_network= FiniteStateAutomaton(n_states=10)
test_node_network.state_space, test_node_network.are_connected(test_node_network.state_space[0], test_node_network.state_space[3])
test_graph.network

TypeError: unsupported operand type(s) for +: 'dict' and 'tuple'

**1. Unit tests**

Unit tests are very low level and close to the source of an application. They consist in testing individual methods and functions of the classes, components, or modules used by your software. Unit tests are generally quite cheap to automate and can run very quickly by a continuous integration server.



**2. Integration tests**

Integration tests verify that different modules or services used by your application work well together. For example, it can be testing the interaction with the database or making sure that microservices work together as expected. These types of tests are more expensive to run as they require multiple parts of the application to be up and running.



In [ ]:
@dataclass
class RandomPolicy:
  action_dims: tuple
  control_magnitude: float

  def __call__(self, observation: np.ndarray):
    return self.control_magnitude* (2*np.random.ranf(self.action_dims)-1)
    
@dataclass
class RandomController(MDPController):
  control_magnitude: float
  
  @property
  def policy(self):
    return RandomPolicy(action_dims=self.environment.action_dims, control_magnitude=self.control_magnitude)
  
  @enforce_method_typing
  def act(self, observation: np.ndarray)-> np.ndarray:
      action= self.policy(observation)
      return action
  
  def observe(self, state)-> np.ndarray:
    observation= state.vector()
    return observation
  
  @enforce_method_typing
  def sample_trajectory(self, runtime: float, n_steps: int=100):
    trajectory= []
    trajectory_return= 0.0
    time= 0.0
    state= self.environment.initial_state
    time_interval= runtime/n_steps
    time_span = np.linspace(time, runtime, n_steps)
    for _ in time_span:
        observation= self.observe(state)
        trajectory.append(observation)
        action= self.act(observation)
        state, reward, _= self.environment.transition_step(state, action, time_interval)
        trajectory_return += reward
    return trajectory, trajectory_return
  
  def plot_trajectory(self, trajectory):
    trajectory= np.array(trajectory)
    px, py, vx, vy= trajectory.transpose()
    plt.figure(figsize=(8, 8))
    plt.plot(px, py, label='Trajectory')
    plt.scatter(px[0], py[0], c='k', marker='o', label='Start')
    plt.scatter(px[-1], py[-1], c='r', marker='*', label='End')
    xmax= max(abs(px))
    ymax= max(abs(py))
    true_max= max((xmax, ymax))
    plt.xlim(-2*true_max, 2*true_max)
    plt.ylim(-2*true_max, 2*true_max)
    plt.grid(True)
    plt.legend()
    plt.show()


**3. Functional tests**
Functional tests focus on the application requirements of the code. Functional tests are performed to check if this module functions as intended.They only verify the output of an action and do not check the intermediate states of the system when performing that action. 

**4. Performance testing**
Performance tests help to measure the reliability, speed, scalability, and responsiveness of an application. It can determine if an application meets performance requirements, locate bottlenecks, measure stability during peak traffic, and more.